This function computes the gradient of the velocity-field $ \mathbf{v}(\mathbf{x}, t) $ using an auxiliary meshgrid.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (3,) | $ \mathbf{x} $ |
| X | array (Ny, Nx, Nz) | X-meshgrid|
| Y | array (Ny, Nx, Nz) | Y-meshgrid|
| Z | array (Ny, Nx, Nz) | Y-meshgrid|
| Interpolant_u | interpolant |  Interpolant object for $ u(\mathbf{x}, t)$|
| Interpolant_v | interpolant |  Interpolant object for $ v(\mathbf{x}, t)$|
| Interpolant_w | interpolant |  Interpolant object for $ w(\mathbf{x}, t)$|
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in z|
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| dt_data | float | time spacing of the velocity data |
| delta | list (3,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data <br /> delta[2]: z-grid spacing of the velocity data|
| aux_grid | float | ratio of auxiliary meshgrid to original meshgrid |
| grad_vel | array (3,3) | $ \nabla \mathbf{v}(\mathbf{x}, t) $ |

In [1]:
# import sys
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add 'integration' folder to current working path
sys.path.append(parent_directory+"/integration")

In [2]:
# Import numpy as np
import numpy as np

# Import function to compute velocity
from ipynb.fs.defs.velocity import velocity

In [3]:
# Import function to compute velocity
from ipynb.fs.defs.velocity import velocity

def gradient_velocity(t, x, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady, aux_grid):
    '''This function computes the gradient of the velocity-field by finite differencing on an auxiliary grid.

    Parameters:
        t: float, time
        x: array (3,Npoints),  vector of positions
        X: array (Ny, Nx, Nz), X-meshgrid
        Y: array (Ny, Nx, Nz), Y-meshgrid
        Z: array (Ny, Nx, Nz), Z-meshgrid
        Interpolant_u: Interpolant object for u(x,t)
        Interpolant_v: Interpolant object for v(x,t)
        Interpolant_w: Interpolant object for w(x,t)
        periodic: list (3,), periodic[0]: periodicity in x
                             periodic[1]: periodicity in y
                             periodic[2]: periodicity in z
        bool_unsteady: bool, specifies if velocity field is unsteady/steady
        dt_data: float, time spacing of the velocity data
        delta: list (3,), delta[0]: x-grid spacing of the velocity data
                          delta[1]: y-grid spacing of the velocity data
                          delta[2]: z-grid spacing of the velocity data
        aux_grid: float, ratio of auxiliary meshgrid to original meshgrid
        
    Returns:
        grad_vel: array (3,3, Npoints),  gradient (Jacobian matrix) of the velocity field
    '''
    x = x.reshape(3,-1)
    
    # define auxiliary grid spacing
    rho_x = aux_grid[0] # float
    rho_y = aux_grid[1] # float
    rho_z = aux_grid[2] # float
    
    X0, XL, XR, XU, XD, XF, XB = [], [], [], [], [], [], []
    
    for i in range(x.shape[1]):
        
        xr = x[0, i] + rho_x # float
        xl = x[0, i] - rho_x # float
         
        if periodic[0]:
            
            xr = (xr-X[0,0,0])%(X[0,-1,0]-X[0,0,0])+X[0,0,0] # float
            xl = (xl-X[0,0,0])%(X[0,-1,0]-X[0,0,0])+X[0,0,0] # float
        
        yu = x[1, i] + rho_y # float
        yd = x[1, i] - rho_y # float
        
        if periodic[1]:
            
            yu = (yu-Y[0,0,0])%(Y[-1,0,0]-Y[0,0,0])+Y[0,0,0] # float
            yd = (yd-Y[0,0,0])%(Y[-1,0,0]-Y[0,0,0])+Y[0,0,0] # float
            
        zf = x[2, i] + rho_z # float
        zb = x[2, i] - rho_z # float
        
        if periodic[2]:
            
            zf = (zf-Z[0,0,0])%(Z[0,0,-1]-Z[0,0,0])+Z[0,0,0] # float
            zb = (zb-Z[0,0,0])%(Z[0,0,-1]-Z[0,0,0])+Z[0,0,0] # float
        
        X0.append([x[0,i], x[1,i], x[2,i]])
        XL.append([xl, x[1,i], x[2,i]])
        XR.append([xr, x[1,i], x[2,i]])
        XU.append([x[0,i], yu, x[2,i]])
        XD.append([x[0,i], yd, x[2,i]])
        XF.append([x[0,i], x[1,i], zf])
        XB.append([x[0,i], x[1,i], zb])
    
    X0 = np.array(X0).transpose()
    XL = np.array(XL).transpose()
    XR = np.array(XR).transpose()
    XU = np.array(XU).transpose()
    XD = np.array(XD).transpose()
    XF = np.array(XF).transpose()
    XB = np.array(XB).transpose()
    
    # velocity on the auxiliary meshgrid
    vLend = velocity(t, XL, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady)    
    vRend = velocity(t, XR, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady)     
    vDend = velocity(t, XD, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady)   
    vUend = velocity(t, XU, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady)
    vFend = velocity(t, XF, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady)   
    vBend = velocity(t, XB, X, Y, Z, Interpolant_u, Interpolant_v, Interpolant_w, periodic, bool_unsteady)
    
    grad_vel = np.zeros((3, 3, X0.shape[1]))*np.nan
    
    for i in range(X0.shape[1]):

            grad_vel[0, 0, i] = (vRend[0,i]-vLend[0,i])/(2*rho_x)
            grad_vel[1, 0, i] = (vRend[1,i]-vLend[1,i])/(2*rho_x)
            grad_vel[2, 0, i] = (vRend[2,i]-vLend[2,i])/(2*rho_x) 
            
            grad_vel[0, 1, i] = (vUend[0,i]-vDend[0,i])/(2*rho_y)
            grad_vel[1, 1, i] = (vUend[1,i]-vDend[1,i])/(2*rho_y)
            grad_vel[2, 1, i] = (vUend[2,i]-vDend[2,i])/(2*rho_y)
            
            grad_vel[0, 2, i] = (vFend[0,i]-vBend[0,i])/(2*rho_z)
            grad_vel[1, 2, i] = (vFend[1,i]-vBend[1,i])/(2*rho_z)
            grad_vel[2, 2, i] = (vFend[2,i]-vBend[2,i])/(2*rho_z)
                
    return grad_vel